In [ ]:
using DrWatson
@quickactivate "sst-mot-analysis"

import DrWatson: datadir, srcdir
import DrWatson: @quickactivate

using GH19, TMI, PythonCall, Revise
using CSV, DataFrames, NaNStatistics, NCDatasets

In [ ]:
function get_MOT_and_SST(TMIfile::String, γ::Grid)
    θ = readfield(TMIfile, "θ", γ); #true mean
    volume = cellvolume(γ).tracer 
    area = cellarea(γ).tracer 

    θ̄_MOT = nansum(θ.tracer .* volume) / nansum(volume)
    θ_SST = getsurfaceboundary(θ);
    θ̄_SST = nansum(θ_SST.tracer .* area) / nansum(area)
    return θ̄_MOT, θ̄_SST
end

function get_MOT_and_SST(θ::Field, γ::Grid)
    volume = cellvolume(γ).tracer 
    area = cellarea(γ).tracer 

    θ̄_MOT = nansum(θ.tracer .* volume) / nansum(volume)
    θ_SST = getsurfaceboundary(θ);
    θ̄_SST = nansum(θ_SST.tracer .* area) / nansum(area)
    return θ̄_MOT, θ̄_SST
end

In [ ]:


###### LOAD LGM ###########
TMI_version = "LGM_90x45x33_G14";
A, Alu, γ_lgm, TMIfile, L, B = config(TMI_version); println(sum(cellvolume(γ_lgm))) 
TMIfile = TMI.pkgdatadir("TMI_"*TMI_version*".nc")

θ̄_LGM_MOT, θ̄_LGM_SST = get_MOT_and_SST(TMIfile, γ_lgm);

###### LOAD MODERN GH19 1990s ###########
TMI_version = GH19.TMIversion()
A, Alu, γ_mod, TMIfile, L, B = config(TMI_version); 

OPT15_ds = NCDataset(datadir("Theta_OPT-0015.nc"))
idx_90s = findmin(abs.(OPT15_ds["year"][:] .- 1990))[2]
GH19_theta = GH19.readfield_snapshot(datadir("Theta_OPT-0015.nc"), "theta", idx_90s, γ_mod)
θ̄_mod_MOT_90s, θ̄_mod_SST_fine_90s = get_MOT_and_SST(GH19_theta, γ_mod);

###### LOAD MODERN GH19 15 CE ###########
TMI_version = GH19.TMIversion()
A, Alu, γ_mod, TMIfile, L, B = config(TMI_version); 

OPT15_ds = NCDataset(datadir("Theta_OPT-0015.nc"))
idx_90s = findmin(abs.(OPT15_ds["year"][:] .- 15))[2]
GH19_theta = GH19.readfield_snapshot(datadir("Theta_OPT-0015.nc"), "theta", idx_90s, γ_mod)
θ̄_mod_MOT_15, θ̄_mod_SST_fine_15 = get_MOT_and_SST(GH19_theta, γ_mod);


In [ ]:
θ̄_mod_MOT_15 - θ̄_LGM_MOT

In [ ]:
θ̄_mod_MOT_15

In [ ]:
θ̄_mod_MOT_90s - θ̄_mod_MOT_15

In [ ]:
θ̄_mod_MOT_90s - θ̄_LGM_MOT

In [ ]:
θ̄_mod_SST_fineθ̄_mod_MOT - θ̄_LGM_MOTθ̄_mod_MOT - θ̄_LGM_MOT - θ̄_LGM_SST